In [2]:
import os
import json
import yaml
import torch
import random
import detectron2

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#move def to utils

# Register the data set
def get_dataset_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)
    
    dataset_dicts = []
    for idx, annots in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, annots["filename"])
        height, width = cv.imread(filename).shape[:2]
        
        #Info on the tile .png file
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        #info on annotation
        annotations = annots["regions"]
        objs = []
        for _, anno in annotations.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px),
                         np.min(py),
                         np.max(px),
                         np.max(py)
                        ],
                #XYXY_ABS: https://detectron2.readthedocs.io/en/latest/_modules/detectron2/structures/boxes.html
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": anno["category"]
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

classes = ["asset"]
colors = [(249, 180, 45)]

for d in ["train", "val"]:
    DatasetCatalog.register(d, lambda d=d: get_dataset_dicts(os.path.join("NSO", d)))
    MetadataCatalog.get(d).thing_classes = classes
    MetadataCatalog.get(d).thing_colors = colors



In [ ]:
# Check annotations on train set

metadata = MetadataCatalog.get("train")
dataset_dicts = get_dataset_dicts("NSO/train")

for d in random.sample(dataset_dicts, 3):
    img = cv.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1, instance_mode=ColorMode.IMAGE)
    out = visualizer.draw_dataset_dict(d)
    print(d["file_name"])
    plt.figure(figsize=(20, 20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

In [17]:
# Setting up model configugation for training

# obtain detectron2's default config
cfg = get_cfg()

# add custom component to configuration
# load values from a file
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)
# initiate weights from model zoo
# check points (.pth) = model's parameters and optimizer state to resume training or evalate performance
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
# set learning rate
cfg.SOLVER.BASE_LR = 0.0005
# the more data you put into the GPU memory, the less memory is available for the model
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
# Results directory for (logs, configs, metrics, and model checkpoints)
cfg.OUTPUT_DIR = "NSO/output"
# print configs 
print(cfg.dump())

# Save your configurations for multi-GPU use
with open("NSO/NSOD2cfg.yaml", "w") as file:
    yaml.dump(cfg, file)

In [18]:
# Single GPU Training

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# DefaultTrainer" which contains pre-defined default logic for standard training workflow.
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
#Expect skip loading parameter because: The warning message is expected. Because your dataset has different number of classes from the pre-trained model, a few pre-trained weights have to be skipped.

[02/15 16:13:06 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[02/15 16:13:06 d2.data.build]: Removed 0 images with no usable annotations. 4 images left.
[02/15 16:13:06 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|  building  | 4            |
|            |              |
[02/15 16:13:06 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[02/15 16:13:06 d2.data.build]: Using training sampler TrainingSampler
[02/15 16:13:06 d2.data.common]: Serializing 4 elements to byte tensors and concatenating them all ...
[02/15 16:13:06 d2.data.common]: Serialized dataset takes 0.00 MiB


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[02/15 16:13:11 d2.engine.train_loop]: Starting training from iteration 0


/scistor/ivm/jpl204/miniconda3/envs/py38/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/scistor/ivm/jpl204/miniconda3/envs/py38/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[02/15 18:03:16 d2.utils.events]:  eta: 0:03:14  iter: 87279  total_loss: 0.008208  loss_cls: 0.0001954  loss_box_reg: 0.00145  loss_mask: 0.006016  loss_rpn_cls: 5.649e-05  loss_rpn_loc: 2.746e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:03:18 d2.utils.events]:  eta: 0:03:13  iter: 87299  total_loss: 0.008649  loss_cls: 0.0003952  loss_box_reg: 0.001683  loss_mask: 0.006268  loss_rpn_cls: 8.112e-05  loss_rpn_loc: 2.353e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:03:19 d2.utils.events]:  eta: 0:03:11  iter: 87319  total_loss: 0.008524  loss_cls: 0.0001811  loss_box_reg: 0.001134  loss_mask: 0.006285  loss_rpn_cls: 8.562e-05  loss_rpn_loc: 3.046e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:03:20 d2.utils.events]:  eta: 0:03:10  iter: 87339  total_loss: 0.008186  loss_cls: 0.0005874  loss_box_reg: 0.001287  loss_mask: 0.005534  loss_rpn_cls: 7.498e-05  loss_rpn_loc: 2.959e-05  time: 0.0741  data_time:

[02/15 18:04:05 d2.utils.events]:  eta: 0:02:32  iter: 87899  total_loss: 0.009093  loss_cls: 0.0003068  loss_box_reg: 0.001261  loss_mask: 0.006055  loss_rpn_cls: 8.417e-05  loss_rpn_loc: 2.342e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:04:06 d2.utils.events]:  eta: 0:02:30  iter: 87919  total_loss: 0.008612  loss_cls: 0.0001991  loss_box_reg: 0.001388  loss_mask: 0.006631  loss_rpn_cls: 6.824e-05  loss_rpn_loc: 2.94e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:04:08 d2.utils.events]:  eta: 0:02:29  iter: 87939  total_loss: 0.009231  loss_cls: 0.0004829  loss_box_reg: 0.001358  loss_mask: 0.005984  loss_rpn_cls: 8.446e-05  loss_rpn_loc: 2.507e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:04:09 d2.utils.events]:  eta: 0:02:27  iter: 87959  total_loss: 0.008921  loss_cls: 0.0001679  loss_box_reg: 0.001145  loss_mask: 0.006654  loss_rpn_cls: 8.895e-05  loss_rpn_loc: 2.664e-05  time: 0.0741  data_time:

[02/15 18:04:51 d2.utils.events]:  eta: 0:01:46  iter: 88519  total_loss: 0.007868  loss_cls: 0.0002667  loss_box_reg: 0.001088  loss_mask: 0.006046  loss_rpn_cls: 6.932e-05  loss_rpn_loc: 2.099e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:04:52 d2.utils.events]:  eta: 0:01:44  iter: 88539  total_loss: 0.01041  loss_cls: 0.000537  loss_box_reg: 0.001028  loss_mask: 0.007421  loss_rpn_cls: 0.0001054  loss_rpn_loc: 2.162e-05  time: 0.0741  data_time: 0.0019  lr: 5e-06  max_mem: 1161M
[02/15 18:04:54 d2.utils.events]:  eta: 0:01:43  iter: 88559  total_loss: 0.008621  loss_cls: 0.0002395  loss_box_reg: 0.001384  loss_mask: 0.006476  loss_rpn_cls: 8.879e-05  loss_rpn_loc: 2.568e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:04:55 d2.utils.events]:  eta: 0:01:41  iter: 88579  total_loss: 0.008226  loss_cls: 0.0001905  loss_box_reg: 0.001433  loss_mask: 0.006463  loss_rpn_cls: 6.99e-05  loss_rpn_loc: 1.796e-05  time: 0.0741  data_time: 0

[02/15 18:05:36 d2.utils.events]:  eta: 0:01:02  iter: 89139  total_loss: 0.008958  loss_cls: 0.0004457  loss_box_reg: 0.001443  loss_mask: 0.005955  loss_rpn_cls: 7.537e-05  loss_rpn_loc: 2.388e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:05:38 d2.utils.events]:  eta: 0:01:00  iter: 89159  total_loss: 0.00867  loss_cls: 0.0003719  loss_box_reg: 0.001178  loss_mask: 0.006123  loss_rpn_cls: 5.388e-05  loss_rpn_loc: 2.031e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:05:39 d2.utils.events]:  eta: 0:00:59  iter: 89179  total_loss: 0.00868  loss_cls: 0.0006485  loss_box_reg: 0.001247  loss_mask: 0.006357  loss_rpn_cls: 8.285e-05  loss_rpn_loc: 2.215e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:05:41 d2.utils.events]:  eta: 0:00:57  iter: 89199  total_loss: 0.008986  loss_cls: 0.0003428  loss_box_reg: 0.00127  loss_mask: 0.006587  loss_rpn_cls: 6.865e-05  loss_rpn_loc: 2.417e-05  time: 0.0741  data_time: 0

[02/15 18:06:25 d2.utils.events]:  eta: 0:00:17  iter: 89759  total_loss: 0.009644  loss_cls: 0.0002726  loss_box_reg: 0.001234  loss_mask: 0.006828  loss_rpn_cls: 6.655e-05  loss_rpn_loc: 3.005e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:06:27 d2.utils.events]:  eta: 0:00:15  iter: 89779  total_loss: 0.00882  loss_cls: 0.0005504  loss_box_reg: 0.001365  loss_mask: 0.006251  loss_rpn_cls: 0.0001224  loss_rpn_loc: 2.979e-05  time: 0.0741  data_time: 0.0021  lr: 5e-06  max_mem: 1161M
[02/15 18:06:29 d2.utils.events]:  eta: 0:00:14  iter: 89799  total_loss: 0.008994  loss_cls: 0.0005732  loss_box_reg: 0.001274  loss_mask: 0.006071  loss_rpn_cls: 0.0001004  loss_rpn_loc: 2.642e-05  time: 0.0741  data_time: 0.0020  lr: 5e-06  max_mem: 1161M
[02/15 18:06:30 d2.utils.events]:  eta: 0:00:13  iter: 89819  total_loss: 0.01035  loss_cls: 0.0005275  loss_box_reg: 0.001668  loss_mask: 0.006939  loss_rpn_cls: 7.12e-05  loss_rpn_loc: 2.582e-05  time: 0.0741  data_time: 0

In [5]:
# Verify GPU information
print (torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.device(0)
       , torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

True 1 0 <torch.cuda.device object at 0x15544342a070> NVIDIA A30
Using device: cuda

NVIDIA A30
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


total_loss: This is a weighted sum of the following individual losses calculated during the iteration. By default, the weights are all one.

loss_cls: Classification loss in the ROI head. Measures the loss for box classification, i.e., how good the model is at labelling a predicted box with the correct class.

loss_box_reg: Localisation loss in the ROI head. Measures the loss for box localisation (predicted location vs true location).

loss_rpn_cls: Classification loss in the Region Proposal Network. Measures the "objectness" loss, i.e., how good the RPN is at labelling the anchor boxes as foreground or background.

loss_rpn_loc: Localisation loss in the Region Proposal Network. Measures the loss for localisation of the predicted regions in the RPN.

loss_mask: Mask loss in the Mask head. Measures how "correct" the predicted binary masks are.

For more details on the losses (1) and (2), take a look at the Fast R-CNN paper and the code.

For more details on the losses (3) and (4), take a look at the Faster R-CNN paper and the code.

For more details on the loss (5), take a look at the Mask R-CNN paper and the code.

In [ ]:
#.PY FILE:

def build_evaluator(cfg, dataset_name, output_folder=None):
    """
    Create evaluator(s) for a given dataset.
    This uses the special metadata "evaluator_type" associated with each builtin dataset.
    For your own dataset, you can simply create an evaluator manually in your
    script and do not have to worry about the hacky if-else logic here.
    """
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []
    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
    if evaluator_type == "cityscapes_instance":
        return CityscapesInstanceEvaluator(dataset_name)
    if evaluator_type == "cityscapes_sem_seg":
        return CityscapesSemSegEvaluator(dataset_name)
    elif evaluator_type == "pascal_voc":
        return PascalVOCDetectionEvaluator(dataset_name)
    elif evaluator_type == "lvis":
        return LVISEvaluator(dataset_name, output_dir=output_folder)
    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    elif len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)


class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return build_evaluator(cfg, dataset_name, output_folder)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res


def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


def main(args):
    # Spacenet
    classes = ["asset"]
    
    for d in ["train", "val"]:
        DatasetCatalog.register(d, lambda d=d: get_dataset_dicts(os.path.join("NSO", d)))
        MetadataCatalog.get(d).thing_classes = classes
    
    cfg = setup(args)

    if args.eval_only:
        model = Trainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        if cfg.TEST.AUG.ENABLED:
            res.update(Trainer.test_with_TTA(cfg, model))
        if comm.is_main_process():
            verify_results(cfg, res)
        return res

    """
    If you'd like to do anything fancier than the standard training logic,
    consider writing your own training loop (see plain_train_net.py) or
    subclassing the trainer.
    """
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    if cfg.TEST.AUG.ENABLED:
        trainer.register_hooks(
            [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
        )
    return trainer.train()


if __name__ == "__main__":
    args = default_argument_parser().parse_args()
    print("Command Line Args:", args)

    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )